In [1]:
from cltk import NLP
from cltk.data.fetch import FetchCorpus

In [2]:
corpus_downloader = FetchCorpus(language="grc")
corpus_downloader.import_corpus("grc_text_tesserae")
cltk_nlp = NLP(language='grc')

‎𐤀 CLTK version '1.0.6'.
Pipeline for language 'Ancient Greek' (ISO: 'grc'): `GreekNormalizeProcess`, `GreekStanzaProcess`, `GreekEmbeddingsProcess`, `StopsProcess`, `GreekNERProcess`.


In [3]:
DATA_DIR = '/home/zach/cltk_data/grc/text/grc_text_tesserae/texts/'

### Load Corpa

In [4]:
import os
import re
from tqdm.notebook import tqdm
from collections import defaultdict as dd

files = os.listdir(DATA_DIR)

docs = dd(list)
keys = []

for file in tqdm(files):
    key = os.path.splitext(file)[0]
    keys.append(key)
    filepath = '{}/{}'.format(DATA_DIR, file)
    text = re.sub('<[^<]+>', "", open(filepath, encoding="utf8").read())
    text = re.sub('\n', "", text)
    docs[key] = text

  0%|          | 0/821 [00:00<?, ?it/s]

### Annotate Corpa

In [5]:
data = dd(list)
for key in tqdm(keys[:1]):
    data[key] = cltk_nlp.analyze(text=docs[key])

  0%|          | 0/1 [00:00<?, ?it/s]

/home/zach/anaconda3/envs/python38/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
from pystlm.stlm import STLM
from pystlm.suffixtree import SuffixTree
from pystlm.sequence import Sequence

trie = SuffixTree()

doc = data[keys[0]]

for w in doc.tokens:
    trie.add(w)

trie.update_all_counts()
stlm = STLM(trie)
seq = Sequence()

for t in doc.tokens[:3]:
    seq.push_back(t)

stlm.prob(seq)

1.9351189214768455e-05